In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
res_all <- readRDS( 'finemap_results_figure3.rds' )


In [3]:
## Define six colors
colors6 <- c(
	rgb( 107, 223, 72,  max = 255 ),
	rgb( 121, 213, 117, max = 255 ),
	rgb( 129, 214, 161, max = 255 ),
	rgb( 93,  217, 206, max = 255 ),
	rgb( 168, 226, 242, max = 255 ),
	rgb( 101, 193, 227, max = 255 )
)


In [4]:
cascade_files <- file.path('../cascade', 'cascade.input.files.tsv')
traits <- fread(cascade_files)
dim(traits) %>% print()


[1] 35  4


In [5]:
length(intersect(names(res_all), (traits %>% select(annotation) %>% pull())))


[1] 35

In [6]:
res <- list()
for(trait in intersect(names(res_all), (traits %>% select(annotation) %>% pull()))){
    res[[ trait ]] <- res_all[[ trait ]]
}


In [7]:
## Plot figure 3
n_regions           <- sum( sapply( res, function( x ) nrow( drop_na ( x ) ) ) )
n_signals           <- sum( sapply( res, function( x ) sum( as.numeric( drop_na(x)$n_signals ) ) ) )
n_signal_per_region <- table(
    unlist( 
        lapply( res, function( x )
        {
    	    cut(
	    	    x      = as.numeric( drop_na(x)$n_signals ),
    		    breaks = c( 0, 1, 2, 3, 4, 5, Inf ),
        		labels = c( '1', '2', '3', '4', '5', '6+' )
	        )
        } )
    )
)
n_snps_per_signal   <- table(
	unlist( 
	    lapply( res, function( x )
    	{
	        cut( 
		        x      = unlist( sapply( drop_na(x)$n_snps_per_signal, function( x ) as.numeric( strsplit( x, split = '\\|' )[[ 1 ]] ), USE.NAMES = F ) ),
    		    breaks = c( 0, 1, 5, 10, 20, 50, Inf ),
    	    	labels = c( '1', '2-5', '6-10', '11-20', '21-50', '51+' )
	        )
	    } )
	)
)


In [8]:
n_signal_per_region_max <- list()
for(t in names(res)){
    n_signal_per_region_max[[ t ]] <- max(as.integer(unique(res[[ t ]]$n_signals)), na.rm=T)
}


#### Number of distinct signals per region

In [9]:
n_signal_per_region %>%
as.data.frame(responseName = "n_regions") %>%
rename('n_signals' = 'Var1') %>%
as.matrix() %>%
t()


n_signals,1,2,3,4,5,6+
n_regions,33,276,963,1256,1033,1802


In [10]:
sprintf(
    'Distinct association signals. A single signal at %d regions, and two to %d signals at %d regions across %d traits.',
    n_signal_per_region[1],
    max(unlist(n_signal_per_region_max)),
    sum(n_signal_per_region) - n_signal_per_region[1],
    length(names(res))
)

[1] "Distinct association signals. A single signal at 33 regions, and two to 40 signals at 5330 regions across 35 traits."

#### Number of variants in the credible set with >= 99%% posterior probability

In [11]:
n_snps_per_signal %>%
as.data.frame(responseName = "n_signals") %>%
rename('n_variants' = 'Var1') %>%
as.matrix() %>%
t()


n_variants,1,2-5,6-10,11-20,21-50,51+
n_signals,2547,3862,2510,3199,5578,10157


In [12]:
sprintf(
    'Number of variants in the credible set with >= 99%% posterior probability. In total of %d signals were mapped to a single variant in the credible set across %d traits.',
    n_snps_per_signal[1],
    length(names(res))
)

[1] "Number of variants in the credible set with >= 99% posterior probability. In total of 2547 signals were mapped to a single variant in the credible set across 35 traits."

In [13]:
add_string <- function(str_list, prefix='', suffix='', idxs = NULL){
    if(is.null(idxs)){
        idxs <- c(length(str_list))
    }
    for(idx in idxs){
        str_list[idx] <- paste0(prefix, str_list[idx], suffix)            
    }

    str_list
}

In [14]:
c(1080, 300) / 60

[1] 18  5

In [17]:
# png(file.path('figs', 'fig3.png'), width = 1080, height = 300)
# pdf(file.path('figs', 'fig3.pdf'))
pdf(file.path('figs', 'fig3A.pdf'), width = 12, height = 3)

layout( matrix( 1 : 2 ), height = c( 0.5, 0.5 ) )

## Number of distinct signals per region
# par( mar = c( 2, 0.25, 0.25, 0.25 ) )
par( mar = c( 1, .5, 0.25, 0.5 ) )
plot(
	x    = 0,
	y    = 0,
	type = 'n',
	xlim = c( 0, 1 ),
# 	ylim = c( -0.1, 1.25 ),
	ylim = c( -0.1, 1.5 ),
	xaxs = 'i',
	yaxs = 'i',
	xaxt = 'n',
	yaxt = 'n',
	xlab = '',
	ylab = '',
	bty  = 'n'
)
start_end_rect <- cbind(
	c( 0, cumsum( n_signal_per_region / n_regions )[ -length( n_signal_per_region ) ] ),
	cumsum( n_signal_per_region / n_regions )
)
axis( 1, tick = F, line = -6.3, at = rowMeans( start_end_rect ), labels = add_string( names( n_signal_per_region ), '', '  signals each', c(3,4,5,6) ) ) 
rect( start_end_rect[, 1 ], 0, start_end_rect[, 2 ], 1, col = colors6, border = 'white' )
text( rowMeans( start_end_rect ), rep( 0.5, nrow( start_end_rect ) ), labels = add_string( n_signal_per_region, 'in ', ' regions', c(3,4,5,6) ) )
lines( c( 0, 1 ), rep( -0.05, 2 ) )
lines( rep( 0, 2 ), c( -0.08, -0.02 ) )
lines( rep( 1, 2 ), c( -0.08, -0.02 ) )
legend_str <- sprintf( '%s distinct signals identified in %s regions', n_signals, n_regions )
rect( 0.5 - 0.6 * strwidth( legend_str ), -0.1, 0.5 + 0.6 * strwidth( legend_str ), 0, col = 'white', border = 'white' )
text( 0.5, -0.1, labels = legend_str, xpd = T )
text( 0, 1.375, labels = 'Number of distinct signals per region', pos = 4, font = 2, offset = 0 )

## Number of variants in the credible set with >= 99%% posterior probability
# par( mar = c( 2, 0.25, 0.25, 0.25 ) )
par( mar = c( 1, .5, 0.25, 0.5 ) )
plot(
	x    = 0,
	y    = 0,
	type = 'n',
	xlim = c( 0, 1 ),
	ylim = c( -0.1, 1.5 ),
	xaxs = 'i',
	yaxs = 'i',
	xaxt = 'n',
	yaxt = 'n',
	xlab = '',
	ylab = '',
	bty  = 'n'
)
start_end_rect <- cbind(
	c( 0, cumsum( n_snps_per_signal / n_signals )[ -length( n_snps_per_signal ) ] ),
	cumsum( n_snps_per_signal / n_signals )
)
axis( 1, tick = F, line = -6.3, at = rowMeans( start_end_rect ), labels = add_string( names( n_snps_per_signal ), '', ' variants each', c(5,6) ) )
rect( start_end_rect[, 1 ], 0, start_end_rect[, 2 ], 1, col = colors6, border = 'white' )
text( rowMeans( start_end_rect ), rep( 0.5, nrow( start_end_rect ) ), labels = add_string( n_snps_per_signal, 'in ', ' signals', c(5,6) ) )
lines( c( 0, start_end_rect[ nrow( start_end_rect ) - 1, 2 ] ), rep( -0.05, 2 ) )
lines( rep( 0, 2 ), c( -0.08, -0.02 ) )
lines( rep( start_end_rect[ nrow( start_end_rect ) - 1, 2 ], 2 ), c( -0.08, -0.02 ) )
legend_str <- sprintf( '%s signals fine-mapped to < 51 variants', sum( n_snps_per_signal[ -length( n_snps_per_signal ) ] ) )
mid_pos_legend <- mean( c( 0, start_end_rect[ nrow( start_end_rect ) - 1, 2 ] ) )
rect( mid_pos_legend - 0.6 * strwidth( legend_str ), -0.1, mid_pos_legend + 0.6 * strwidth( legend_str ), 0, col = 'white', border = 'white' )
text( mid_pos_legend, -0.1, labels = legend_str, xpd = T )
text( 0, 1.375, labels = 'Number of variants in each signal', pos = 4, font = 2, offset = 0 )


dev.off()

png 
  2

In [16]:
# options(repr.plot.width=8, repr.plot.height=2)

# n_signal_per_region %>%
# data.frame() %>%
# rename('n_signal' = 'Var1', 'n_loci' = 'Freq') %>%
# mutate(x = '') %>%
# ggplot(aes(x = factor(x), y = n_loci, fill=factor(n_signal, levels=rev(n_signal))) )+
# theme_minimal() +
# coord_flip() +
# geom_bar(position="stack", stat="identity") +
# theme(legend.position="bottom") +
# labs(
#     x = '',
#     y = 'number of distinct signals',
#     fill = 'Number of loci'
# )